# Libraries

In [ ]:
 
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import numpy as np
from tensorflow.keras import optimizers
import tensorflow.keras as keras
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install kaggle
!pip install --upgrade kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c uw-cs480-fall20
!ls
import zipfile
zip_ref = zipfile.ZipFile('uw-cs480-fall20.zip', 'r')
zip_ref.extractall('files')
zip_ref.close()

     |████████████████████████████████| 61kB 3.2MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.10-cp36-none-any.whl size=73269 sha256=22d8e6edfb6e4d124fe2b1b25bec832705958d25729f531c589c154571730c4d
  Stored in directory: /root/.cache/pip/wheels/3a/d1/7e/6ce09b72b770149802c653a02783821629146983ee5a360f10
Successfully built kaggle
  Found existing installation: kaggle 1.5.9
    Uninstalling kaggle-1.5.9:
      Successfully uninstalled kaggle-1.5.9


Saving kaggle.json to kaggle.json
 95% 81.0M/85.6M [00:06<00:00, 9.72MB/s]
100% 85.6M/85.6M [00:06<00:00, 14.6MB/s]
drive  kaggle.json  sample_data  uw-cs480-fall20.zip


# Data Upload

In [ ]:
import pandas as pd
train = pd.read_csv('/content/files/train.csv',dtype=str)
test = pd.read_csv('/content/files/test.csv',dtype=str)
# train=train.reset_index(drop=True)
# test=test.reset_index(drop=True)
def append_ext(fn):
    return fn+".jpg"
train["id"]=train["id"].apply(append_ext)
test["id"]=test["id"].apply(append_ext)   
train_folder = '/content/files/suffled-images/shuffled-images'
test_folder = '/content/files/suffled-images/shuffled-images'

In [ ]:
def feature2Num (train,cat):
  le = preprocessing.LabelEncoder()
  ax=train[cat]
  ax2=pd.Categorical(ax)
  le.fit(ax2)
  ax3 = le.transform(ax2)
  return pd.get_dummies(ax3).to_numpy()

gender = feature2Num(train,"gender")

print(gender.shape)

(21627, 5)


In [ ]:
gender = feature2Num(train,"gender")
baseColour = feature2Num(train,"baseColour")
season = feature2Num(train,"season")
usage = feature2Num(train,"usage")
feature_train = np.concatenate((gender, baseColour, season, usage), axis=1)
feature_train_lable = feature2Num(train,"category")
print(feature_train.shape)

gender = feature2Num(test,"gender")
baseColour = feature2Num(test,"baseColour")
season = feature2Num(test,"season")
usage = feature2Num(test,"usage")
feature_test = np.concatenate((gender, baseColour, season, usage), axis=1)
print(feature_test.shape)
from sklearn.model_selection import train_test_split

(21627, 62)
(21628, 62)


In [ ]:
model2 = keras.models.load_model('/content/drive/MyDrive/my_model_92.h2')

In [ ]:
# Define your data generator
train_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

X_train_img = train_gen.flow_from_dataframe(dataframe = train, 
directory = train_folder, x_col = 'id', 
y_col = 'category', subset="training", seed = 42,
batch_size = 100, shuffle = False, 
class_mode="categorical",target_size = (80, 60))
############################################################
X_test_img = test_gen.flow_from_dataframe(dataframe = test, 
directory = test_folder, x_col = 'id', 
y_col = None,
batch_size = 100, shuffle = False, 
class_mode=None,target_size = (80, 60))

Found 21627 validated image filenames belonging to 27 classes.
Found 21628 validated image filenames.


In [ ]:
vgg_out_train = model2.predict_proba(X_train_img,verbose=1)
vgg_out_test = model2.predict_proba(X_test_img,verbose=1)

Instructions for updating:
Please use `model.predict()` instead.
217/217 [==============================] - 8s 39ms/step


In [ ]:
ax = model2.predict_proba(X_train_img,verbose=1)
print(ax)

217/217 [==============================] - 8s 37ms/step
[[3.7275952e-14 3.0140463e-09 4.1152384e-06 ... 9.8850548e-01
  2.4767660e-14 5.1111118e-07]
 [1.5650393e-12 3.5764143e-13 5.8651779e-07 ... 1.3389703e-07
  2.5506983e-09 7.5390270e-11]
 [1.7618684e-18 8.7691909e-11 2.9406604e-09 ... 9.9999309e-01
  6.1342291e-17 7.9800840e-11]
 ...
 [4.6418008e-23 2.6785230e-34 1.7165532e-13 ... 1.9308075e-14
  2.6774555e-15 1.6746539e-21]
 [7.2769013e-09 2.3566233e-08 3.3816668e-05 ... 2.7687651e-05
  8.0381500e-11 9.9887234e-01]
 [1.1525995e-25 1.3068848e-16 1.0256029e-10 ... 1.0000000e+00
  4.9314772e-22 3.6345747e-13]]


In [ ]:
sentences = train['noisyTextDescription'].values
labels = train['category'].values
# print(labels)
sentences_train = train['noisyTextDescription'].values
sentences_test  = test['noisyTextDescription'].values
y_train = train['category'].values
print(sentences_train)

['Femella Women Ankle-Length Grey AQ-S800WD-1EVDFAD170'
 'Converse Unisex Casual Skirts Slipper' 'Velia Women Acetone Kurta' ...
 'Rocia Women R348 Flats' 'Fastrack Men Black Amethyst Watch'
 'Arrow Ghicha Men White Slim Surf Shirt']


In [ ]:
model3 = keras.models.load_model('/content/drive/MyDrive/MLP.h5')

In [ ]:
text_out_train = model3.predict_proba(sentences_train,verbose=1)
print(text_out_train[0])

text_out_test = model3.predict_proba(sentences_test,verbose=1)
print(text_out_test[0])


ValueError: ignored

In [ ]:
print(vgg_out_train.shape)
print(feature_train.T.shape)
print(vgg_out_test.shape)
print(feature_test.T.shape)
X_train_conc=np.concatenate((vgg_out_train,feature_train), axis=1)
X_test_conc=np.concatenate((vgg_out_test,feature_test),axis=1)
print(X_train_conc.shape)
print(X_test_conc.shape)
print(feature_train_lable.shape)


(21627, 27)
(62, 21627)
(21628, 27)
(62, 21628)
(21627, 89)
(21628, 89)
(21627, 27)


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train_conc, feature_train_lable, test_size=0.05, random_state=0)
# y_train = pd.get_dummies(y_train).to_numpy()
# y_val = pd.get_dummies(y_val).to_numpy()
X_test = X_test_conc;
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)

(20545, 89)
(1082, 89)
(20545, 27)


In [ ]:
model = Sequential()
model.add(Dense(1024, input_dim=X_train.shape[1], activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(Dense(512, input_dim=X_train.shape[1], activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(Dense(27))
model.add(Activation('softmax'))

opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=200, batch_size=50, verbose=1)
# evaluate the keras model
_, accuracy = model.evaluate(X_val, y_val, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))


Epoch 1/200
411/411 [==============================] - 1s 3ms/step - loss: 0.5975 - accuracy: 0.8599 - val_loss: 0.1633 - val_accuracy: 0.9649
Epoch 2/200
411/411 [==============================] - 1s 2ms/step - loss: 0.2042 - accuracy: 0.9580 - val_loss: 0.1378 - val_accuracy: 0.9704
Epoch 3/200
411/411 [==============================] - 1s 2ms/step - loss: 0.1671 - accuracy: 0.9660 - val_loss: 0.1327 - val_accuracy: 0.9695
Epoch 4/200
411/411 [==============================] - 1s 2ms/step - loss: 0.1481 - accuracy: 0.9690 - val_loss: 0.1224 - val_accuracy: 0.9732
Epoch 5/200
411/411 [==============================] - 1s 2ms/step - loss: 0.1380 - accuracy: 0.9702 - val_loss: 0.1241 - val_accuracy: 0.9723
Epoch 6/200
411/411 [==============================] - 1s 2ms/step - loss: 0.1276 - accuracy: 0.9724 - val_loss: 0.1269 - val_accuracy: 0.9741
Epoch 7/200
411/411 [==============================] - 1s 2ms/step - loss: 0.1213 - accuracy: 0.9733 - val_loss: 0.1296 - val_accuracy: 0.9732

In [ ]:
pred_vgg=model.predict_generator(X_test,verbose=1)
predicted_class_indices_vgg=np.argmax(pred_vgg,axis=1)

labels = (X_train_img.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions_vgg = [labels[k] for k in predicted_class_indices_vgg]
filenames=X_test_img.filenames
results_vgg=pd.DataFrame({"id":filenames,"category":predictions_vgg})
results_vgg.to_csv("results_rmsp_vgg_hot.csv",index=False)

676/676 [==============================] - 1s 891us/step


In [ ]:
ax = model.save_weights

In [ ]:
model.save('my_model_91.h2')
model2 = keras.models.load_model('my_model_91.h2')
print(model2.predict_generator(test_data,verbose=1))

In [ ]:
print(ax)